In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 70.4 MB/s eta 0:00:00


In [6]:
#!pip install openai langchain

#Registro con API de OpenAI

In [2]:
import openai
import getpass

api_key = getpass.getpass("Enter your OpenAI API Key:")

client = openai.OpenAI(api_key=api_key)

Enter your OpenAI API Key:··········


# Creación de un ChatBot de Horticultura a partir de un modelo pre-entrenado de OpenAI.

La siguiente función crea un prompt y lo modifica para que cuando se lo pasemos al modelo, este actue como un experto en horticultura.
Tiene dos ejemplos como técnica de few-shot.

In [3]:
def crear_prompt(pregunta_usuario):
    prompt = f"""
    Eres un experto en horticultura y agricultura ecológica.
    Responde con consejos claros y prácticos sobre cómo cuidar plantas y cultivos.
    No inventes si no estás seguro, y si la pregunta no tiene sentido, dilo.
    ---
    Pregunta del usuario: ¿Cuando propagar Hydrangeas?
    Respuesta: En verano. Cortar esquejes con dos a tres nodos, entierra parte del esqueje en una maceta con sustrato universal y corta las hojas del esqueje por la mitad. En unas semanas el esqueje echará raices
    ---
    Pregunta: ¿Qué tipo de suelo necesita la lavanda?
    Respuesta: Suelo arenoso, con buen drenaje y pobre en nutrientes. No le gusta la humedad excesiva.

    Pregunta del usuario: {pregunta_usuario}
    """
    return prompt

In [15]:
pregunta = '¿Cómo cuido un limonero en maceta en invierno?'
def get_completion(prompt, model="gpt-3.5-turbo", temperature = 0):
  respuesta = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "Eres un asistente experto en cultivos. Responde de forma técnica pero sencilla"},
      {"role": "user", "content": crear_prompt(prompt)}
    ]
  )
  return(respuesta.choices[0].message.content)
  print(get_completion(prompt=pregunta))

In [6]:
pregunta = 'Cómo se crian caracoles?'
respuesta_caracoles = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Eres un asistente experto en cultivos. Responde de forma técnica pero sencilla"},
    {"role": "user", "content": crear_prompt(pregunta)}
  ]
)
print(respuesta_caracoles.choices[0].message.content)

Lo siento, criar caracoles no es una práctica común ni recomendada en horticultura o agricultura. Si tienes más preguntas sobre el cuidado de plantas y cultivos, estaré encantado de ayudarte. ¡Gracias!


#ChatBot de horticultura que combine información propia de su entrenamiento de LLM con información adicional obtenida de un RAG.

* Primero, cargamos y procesamos el texto que constituira el RAG. Para ello, dirigete al notebook de `RAG_preprocessing.ipynb` y ejecutalo. Esto creará y guardará los chunks, embeddings e índice a usar para el RAG.

* Después, volver a este notebook y ejecutar esta segunda parte.

In [7]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import torch

#from google.colab import drive
#drive.mount('/content/drive')

ruta_base = "/content/drive/MyDrive/Bootcamp/IA/"
# Cargar chunks y embeddings
chunks = np.load(f"{ruta_base}horticultura_chunks.npy", allow_pickle=True)
index = faiss.read_index(f"{ruta_base}horticultura_index.faiss")

# Función búsqueda de RAG
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [29]:
def get_completion_con_rag(pregunta):
    # 1. Búsqueda semántica con filtro de calidad
    query_embedding = model.encode([pregunta], normalize_embeddings=True)
    D, I = index.search(query_embedding.astype('float32'), k=3)
    chunks_relevantes = [chunks[i] for i, score in zip(I[0], D[0]) if score > 0.5]

    # 2. Decide si usar RAG o solo LLM
    if not chunks_relevantes:
        respuesta = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": crear_prompt(pregunta)}]
        ).choices[0].message.content
        return f"[Fuente: Conocimiento general del modelo LLM]\n\n{respuesta}"

    # 3. Respuesta aumentada con RAG
    contexto = ' '.join(chunks_relevantes)
    prompt = f"""
    [CONTEXTO DOCUMENTAL]
    {contexto}
    [INSTRUCCIÓN]
    Responde ÚNICAMENTE usando la información del contexto anterior.
    Pregunta: {pregunta}
    """

    respuesta = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Eres un asistente que responde estrictamente basado en el documento proporcionado."},
            {"role": "user", "content": prompt}
        ]
    )

    # Añade referencia a los fragmentos usados
    referencias = "\n\nFragmentos consultados:\n" + "\n".join(f"- {chunk[:100]}..." for chunk in chunks_relevantes)
    return f"[Fuente: Documentos procesados (RAG)]\n\n{respuesta.choices[0].message.content}{referencias}"

In [26]:
print(get_completion_con_rag("¿Qué es el monocultivo?"))

[Fuente: Conocimiento general del modelo LLM]

Respuesta: El monocultivo es un sistema de agricultura en el que se cultiva una sola especie de planta en una gran extensión de terreno. Esto puede agotar los nutrientes del suelo y aumentar la susceptibilidad a plagas y enfermedades. Es importante diversificar los cultivos para mantener la salud del suelo y la biodiversidad.


In [32]:
print(get_completion_con_rag("¿How do you control aphids?"))

[Fuente: Documentos procesados (RAG)]

The peach-potato aphid is controlled in three main ways. In outdoor crops, several organisms are involved.

Fragmentos consultados:
- digestive system. Fumigant chemicals, e.g. nicotine, enter the insect through the spiracles. There a...
- the aphid stylets may transmit viruses suchas virus Y on potatoes and tomato aspermy viruson chrysan...
- life cycle stages. There follows a selection of insect pests in which each species has particular fe...


In [31]:
print(get_completion_con_rag("¿What are some annual weeds?"))

[Fuente: Documentos procesados (RAG)]

Some annual weeds mentioned in the context are chickweed and black nightshade.

Fragmentos consultados:
- sterilized. Below are described important annual and perennial weeds. Speciﬁc descriptions of identi...
- measurement, 24–27Weathering, 152–153 biological, 153 chemical, 153 physical, 153 Weed(s), 87–96, 11...
- leaves that pre-vent light reaching emergent plant seedlings, while groundsel and fat hen have an up...


<image>

# Conclusiones
* El ChatBot solo consulta el documento RAG cuando las consultas son en inglés, como es de esperar. Para consultas en español o poco relaccionadas con el documento proporcionado, usar el conocimiento del LLM.
* El preprocesado del documento para el RAG y creación de los chunks, podría haber sido más elaborado y meticuloso, pero como prueba concepto, el documento funciona.
* El proyecto tiene la posibilidad de incrementar su conocimiento del RAG a partir de otros documentos añadidos y procesados para hacer que el ChatBot se especialice aún más en ciertos campos.